# Fixing a base directory for all operations

In [2]:
!pip install bs4 nltk numpy vaderSentiment

In [3]:
import os,json,io
print ('Getting base directory... ', end='')
base = os.getcwd()
print ('Done\nSetting the base at', base)

Getting base directory... Done
Setting the base at /home/jovyan


# Listing all avilable matches in commentary database

In [4]:
from bs4 import BeautifulSoup as bsp
print ('Changing to the commentary db folder', end='')
os.chdir(os.path.join(base,'commentaries'))
print (os.getcwd(),'... Done')

print('Collecting all matches in current folder...', end='')
matches = [f for f in os.scandir(os.getcwd()) if f.is_dir() and f.name.isdigit()];
print(' Done')

Changing to the commentary db folder/home/jovyan/commentaries ... Done


### These are avilable matches' id.
Saving them as DirEntry to use a few feature of this object type

In [5]:
#test
matches

[<DirEntry '1153635'>,
 <DirEntry '1146722'>,
 <DirEntry '1153636'>,
 <DirEntry '1144487'>,
 <DirEntry '1153316'>,
 <DirEntry '1153315'>,
 <DirEntry '1146721'>,
 <DirEntry '1130741'>,
 <DirEntry '1153255'>,
 <DirEntry '1130737'>,
 <DirEntry '1130739'>,
 <DirEntry '1153637'>,
 <DirEntry '1153252'>,
 <DirEntry '1146720'>,
 <DirEntry '1153254'>,
 <DirEntry '1130743'>,
 <DirEntry '1168514'>,
 <DirEntry '1130742'>,
 <DirEntry '1168509'>,
 <DirEntry '1153314'>,
 <DirEntry '1144491'>,
 <DirEntry '1168513'>,
 <DirEntry '1153248'>,
 <DirEntry '1168512'>,
 <DirEntry '1153243'>,
 <DirEntry '1153846'>,
 <DirEntry '1153845'>,
 <DirEntry '1153844'>,
 <DirEntry '1153249'>]

In [6]:
total_commentary = 0
total_sent = 0

In [7]:
#putting frequently used text in variable, helpful in IDE's with auto completion
t_commentary = 'commentary'
t_items = 'items'
t_shorttext = 'shortText'
t_postText = 'postText'
t_text = 'text'
t_preText = 'preText'

#functions I'll need
#-----------------------------------------get_match_details-----------------------------------------------------
def get_match_details(match_id):
    import requests
    r = requests.get(f'http://www.espncricinfo.com/series/engine/match/{match_id}.json')
    return r.json()
#---------------------------------------------------------------------------------------------------------------
def get_required_innings_value(country):
    pass
    
def get_vader_sentence_score(sentence_string):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

    scores = SentimentIntensityAnalyzer()

    return scores.polarity_scores(sentence_string)

#----------------------------------------------------------------------------------------------------------------------------
def get_all_text_fromjsonfile(data):
    global total_commentary
    alltext = ''
    bslash = '\\'
    for item in data[t_commentary][t_items]:
        total_commentary += 1
        alltext += item[t_preText] + item[t_postText]
    return bsp(alltext, 'html.parser').text
#--------------------------------------------------------------------------------------------------------------------
def get_player_score_fromjson(data):
    import nltk, io
    from nltk.tokenize import sent_tokenize, word_tokenize
    from nltk.corpus import stopwords

    def without_stopwords(word_list):
        stopset = set(stopwords.words('english'))
        return [x for x in word_list if x not in stopset]

    players = dict()
    
    with io.open(os.path.join(base,'player_names.json'), 'r') as fp:
        players = json.load(fp)
#         print('\n', players,'\n')
        for sent in nltk.sent_tokenize(get_all_text_fromjsonfile(data)):
#             print (sent)
            for chunk in nltk.ne_chunk(nltk.pos_tag(without_stopwords(nltk.word_tokenize(sent)))):
                if hasattr(chunk, 'label') and (chunk.label() == 'PERSON'):
                    playerFromSent = ' '.join(c[0] for c in chunk)
#                     print(f'\nworking with {playerFromSent}\n')
                    for player in players.keys():
                        if playerFromSent.lower() in players[player]:
#                             print (playerFromSent,'found\n')
                            personal_comm.setdefault(player.lower(),[]).append([get_vader_sentence_score(sent)['compound'],sent])
    return personal_comm

##### We have made a function that evaluates a sentence.  (get_player_score_fromjson)
Next, we'll put a candidate sentence through it and get a result
### Okay, for simplicity let's work with <i style="color : red">postText</i>, <i style="color : red">preText</i> and give a scoring based on that
- For each sentence we find a player's name we shall repeat this process.

In [8]:
import nltk
nltk.download("punkt")
nltk.download("brown")
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download("words")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/jovyan/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# <span style="color:red;">Let's Begin!</span>

In [9]:
personal_comm = dict()
for match in matches:
    os.chdir(match.path);
    #match_details = get_match_details(match.name)
    print ("moving to "+match.path);
    innings = [f.path for f in os.scandir(match.path) if f.is_dir()]
    for single_innings in innings: #There is two innings in one one day match, we'll iterate foe both of them
        os.chdir(single_innings);
        print ("moving to "+single_innings);
        for jsonfile in os.listdir():
            if 'comm_part' in jsonfile: #comm_part is a common substring in the filename of json files. this condition prevents other files from hampering file iteration
                print("opening " + jsonfile)
                with io.open(jsonfile, 'r') as file:
                    data = json.load(file)
                    get_player_score_fromjson(data) #passing the file contents to the previously written function
                    
#                     print(metadata)
# personal_comm

moving to /home/jovyan/commentaries/1153635
moving to /home/jovyan/commentaries/1153635/innings02
opening comm_part13.json
opening comm_part10.json
opening comm_part1.json
opening comm_part2.json
opening comm_part11.json
opening comm_part4.json
opening comm_part6.json
opening comm_part3.json
opening comm_part5.json
opening comm_part9.json
opening comm_part12.json
opening comm_part7.json
opening comm_part8.json
moving to /home/jovyan/commentaries/1153635/innings01
opening comm_part13.json
opening comm_part10.json
opening comm_part1.json
opening comm_part2.json
opening comm_part11.json
opening comm_part4.json
opening comm_part6.json
opening comm_part3.json
opening comm_part5.json
opening comm_part9.json
opening comm_part12.json
opening comm_part7.json
opening comm_part8.json
moving to /home/jovyan/commentaries/1146722
moving to /home/jovyan/commentaries/1146722/innings02
opening comm_part13.json
opening comm_part10.json
opening comm_part1.json
opening comm_part2.json
opening comm_part11.

opening comm_part2.json
opening comm_part11.json
opening comm_part4.json
opening comm_part6.json
opening comm_part3.json
opening comm_part5.json
opening comm_part9.json
opening comm_part7.json
opening comm_part8.json
moving to /home/jovyan/commentaries/1153637/innings01
opening comm_part13.json
opening comm_part10.json
opening comm_part1.json
opening comm_part2.json
opening comm_part11.json
opening comm_part4.json
opening comm_part6.json
opening comm_part3.json
opening comm_part5.json
opening comm_part9.json
opening comm_part12.json
opening comm_part7.json
opening comm_part8.json
moving to /home/jovyan/commentaries/1153252
moving to /home/jovyan/commentaries/1153252/innings02
opening comm_part13.json
opening comm_part10.json
opening comm_part1.json
opening comm_part2.json
opening comm_part11.json
opening comm_part4.json
opening comm_part6.json
opening comm_part3.json
opening comm_part5.json
opening comm_part9.json
opening comm_part12.json
opening comm_part7.json
opening comm_part8.json

opening comm_part2.json
opening comm_part11.json
opening comm_part4.json
opening comm_part6.json
opening comm_part3.json
opening comm_part5.json
opening comm_part9.json
opening comm_part12.json
opening comm_part7.json
opening comm_part8.json
moving to /home/jovyan/commentaries/1168512/innings01
opening comm_part13.json
opening comm_part10.json
opening comm_part1.json
opening comm_part2.json
opening comm_part11.json
opening comm_part4.json
opening comm_part6.json
opening comm_part3.json
opening comm_part5.json
opening comm_part9.json
opening comm_part12.json
opening comm_part7.json
opening comm_part8.json
moving to /home/jovyan/commentaries/1153243
moving to /home/jovyan/commentaries/1153243/innings02
opening comm_part1.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part3.json
opening comm_part5.json
opening comm_part9.json
opening comm_part7.json
opening comm_part8.json
moving to /home/jovyan/commentaries/1153243/innings01
opening comm_part13.

In [19]:
total_commentary

15968

In [11]:
total_sent

0

In [12]:
# match.path #this is random variable checking. nothing important

# Next Step
#### Create a function that takes in commetary json and spits out a dictionary that contains player names and commentary sentences about them

    - have to add the save_player_names to this module

In [13]:
os.getcwd()

'/home/jovyan/commentaries/1153249/innings01'

### Saving commentary, related player names, and scores for each sentence for that player

In [14]:
with open(os.path.join(base,'result-json2.json'), 'w') as fp:
    json.dump(personal_comm,fp)

In [15]:
os.getcwd() #checking which directory I am at right now

'/home/jovyan/commentaries/1153249/innings01'

# Calculating a mean value for each player

In [16]:
result = dict()

for key in personal_comm.keys():
    sumofpoints = 0
    for item in personal_comm[key]:
        total_sent += 1
        sumofpoints += item[0]
#     print (key, sumofpoints, len(personal_comm[key]))
    result[key] = sumofpoints/len(personal_comm[key])

In [17]:
total_sent


980

In [18]:
for player in result.keys():
    print(f"\\item {player.title()} - {len(personal_comm[player])} times")

\item Imrul Kayes - 70 times
\item Mohammad Saifuddin - 30 times
\item Mustafizur Rahman - 28 times
\item Mashrafe Mortaza - 126 times
\item Shakib Al Hasan - 144 times
\item Liton Das - 86 times
\item Mehidy Hasan - 54 times
\item Tamim Iqbal - 141 times
\item Mushfiqur Rahim - 74 times
\item Mohammad Mithun - 35 times
\item Rubel Hossain - 45 times
\item Mahmudullah - 26 times
\item Soumya Sarkar - 85 times
\item Najmul Hossain Shanto - 4 times
\item Anamul Haque - 15 times
\item Nasir Hossain - 10 times
\item Taskin Ahmed - 7 times


### Sorting the dictionary for comparative player ranking

In [41]:
import operator
sorted_list = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [45]:
personal_comm.keys()

dict_keys(['imrul kayes', 'mohammad saifuddin', 'mustafizur rahman', 'mashrafe mortaza', 'shakib al hasan', 'liton das', 'mehidy hasan', 'tamim iqbal', 'mushfiqur rahim', 'mohammad mithun', 'rubel hossain', 'mahmudullah', 'soumya sarkar', 'najmul hossain shanto', 'anamul haque', 'nasir hossain', 'taskin ahmed'])

In [42]:
sorted_list

[('mashrafe mortaza', 0.27430952380952384),
 ('mehidy hasan', 0.23445000000000002),
 ('soumya sarkar', 0.2078494117647059),
 ('imrul kayes', 0.20449428571428566),
 ('mohammad saifuddin', 0.1850766666666667),
 ('mohammad mithun', 0.1814057142857143),
 ('najmul hossain shanto', 0.176),
 ('tamim iqbal', 0.16256170212765947),
 ('liton das', 0.1546546511627907),
 ('shakib al hasan', 0.1383062500000001),
 ('mushfiqur rahim', 0.13402837837837842),
 ('mustafizur rahman', 0.12883571428571428),
 ('mahmudullah', 0.12602692307692306),
 ('rubel hossain', 0.07353777777777779),
 ('taskin ahmed', 0.046271428571428574),
 ('nasir hossain', 0.03095),
 ('anamul haque', -0.05986666666666666)]

## Reversing the key-value to value-key

In [13]:
newdict = dict()
for x,y in sorted_list:
    newdict[y] = x

In [14]:
val_list = [x for y,x in sorted_list ]

##  <span style="color : limegreen">Selected players and their points</span>
Printing top eleven player due to their points
*val_list already contains a sorted list of points that are assigned to each player*

In [15]:
for x in val_list[:11]:
    print (newdict[x], x)

mashrafe mortaza 0.22858319999999988
mehidy hasan 0.1963518518518519
mohammad saifuddin 0.15109333333333336
soumya sarkar 0.15051882352941176
imrul kayes 0.13948260869565218
mohammad mithun 0.12819142857142862
tamim iqbal 0.11337163120567378
mustafizur rahman 0.11148928571428572
liton das 0.10149651162790697
mushfiqur rahim 0.09308783783783783
shakib al hasan 0.08590555555555557


In [9]:
base_db = os.path.join(base, 'commentaries')

# Let's check if the results are accurate or not
### At first let's make lists of 5 matches, 10 matches, 20 matches

In [8]:
match_5 = ['1168512','1168513','1168514', '1144487', '1168509']
match_10 = ['1168512','1168513','1168514', '1144487', '1168509', '1153844', '1153845', '1153846', '1153316', '1153315']
match_20 = ['1168512','1168513','1168514', '1144487', '1168509', '1153844', '1153845', '1153846', '1153316', '1153315', '1153314', '1153635', '1153636', '1153637', '1153255', '1153254', '1153252', '1153249', '1153248', '1153243']

**Following three cells just prepare folder location of relevent number of matches for accuracy testing**

In [18]:
match_20_dir = []
for ent in match_20:
    match_20_dir.append(os.path.join(base_db, ent))
   

In [19]:
match_10_dir = []
for ent in match_10:
    match_10_dir.append(os.path.join(base_db, ent))
   

In [20]:
match_5_dir = []
for ent in match_5:
    match_5_dir.append(os.path.join(base_db, ent))
   

In [21]:
os.chdir(base)

In [22]:
match_10_dir


['/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168513',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168514',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1144487',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168509',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153844',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153845',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153846',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153316',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153315']

### We'll check for a five match squad prediction and compare it with 6th match's squad

In [ ]:
personal_comm = dict()
for match in match_5_dir:
    os.chdir(match);
    #match_details = get_match_details(match.name)
    print ("moving to "+match);
    innings = [f.path for f in os.scandir(match) if f.is_dir()]
    for single_innings in innings:
        os.chdir(single_innings);
        print ("moving to "+single_innings);
        for jsonfile in os.listdir():
            if 'comm_part' in jsonfile:
                print("opening " + jsonfile)
                with io.open(jsonfile, 'r') as file:
                    data = json.load(file)
                    get_player_score_fromjson(data)
                    
#                     print(metadata)
personal_comm

In [24]:
result = dict()
# sumofpoints = 0
for key in personal_comm.keys():
    sumofpoints = 0
    for item in personal_comm[key]:
        sumofpoints += item[0]
#     print (key, sumofpoints, len(personal_comm[key]))
    result[key] = sumofpoints/len(personal_comm[key])

In [25]:
import operator
sorted_list = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [26]:
newdict = dict()
for x,y in sorted_list:
    newdict[y] = x

In [27]:
val_list = [x for y,x in sorted_list ]

## Selected players and their points <span style="color : darkblue"> for 5 matches</span>

In [28]:
for x in val_list[:11]:
    print (newdict[x], x)

liton das 0.3019090909090909
mashrafe mortaza 0.29534400000000005
mehidy hasan 0.16267
mahmudullah 0.128575
najmul hossain shanto 0.128
shakib al hasan 0.1208655172413793
mushfiqur rahim 0.09572142857142855
tamim iqbal 0.05196190476190475
taskin ahmed 0.046271428571428574
mohammad saifuddin 0.04349999999999998
mustafizur rahman 0.030355555555555556


### We'll check for a ten match squad prediction and compare it with 11th match's squad

In [ ]:
personal_comm = dict()
for match in match_10_dir:
    os.chdir(match);
    #match_details = get_match_details(match.name)
    print ("moving to "+match);
    innings = [f.path for f in os.scandir(match) if f.is_dir()]
    for single_innings in innings:
        os.chdir(single_innings);
        print ("moving to "+single_innings);
        for jsonfile in os.listdir():
            if 'comm_part' in jsonfile:
                print("opening " + jsonfile)
                with io.open(jsonfile, 'r') as file:
                    data = json.load(file)
                    get_player_score_fromjson(data)
                    
#                     print(metadata)
personal_comm

In [30]:
result = dict()
# sumofpoints = 0
for key in personal_comm.keys():
    sumofpoints = 0
    for item in personal_comm[key]:
        sumofpoints += item[0]
#     print (key, sumofpoints, len(personal_comm[key]))
    result[key] = sumofpoints/len(personal_comm[key])

In [31]:
import operator
sorted_list = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [32]:
newdict = dict()
for x,y in sorted_list:
    newdict[y] = x

In [33]:
val_list = [x for y,x in sorted_list ]

## Selected players and their points <span style="color : darkblue"> for 10 matches</span>

In [34]:
for x in val_list[:11]:
    print (newdict[x], x)

mashrafe mortaza 0.19364565217391305
mohammad mithun 0.15626666666666666
mohammad saifuddin 0.1513875
mehidy hasan 0.14847200000000002
liton das 0.13023846153846153
najmul hossain shanto 0.128
soumya sarkar 0.11219130434782607
rubel hossain 0.10597368421052632
imrul kayes 0.09156666666666667
tamim iqbal 0.08879999999999999
shakib al hasan 0.05472954545454545


### We'll check for a twenty match squad prediction and compare it with 21st match's squad

In [ ]:
personal_comm = dict()
for match in match_20_dir:
    os.chdir(match);
    #match_details = get_match_details(match.name)
    print ("moving to "+match);
    innings = [f.path for f in os.scandir(match) if f.is_dir()]
    for single_innings in innings:
        os.chdir(single_innings);
        print ("moving to "+single_innings);
        for jsonfile in os.listdir():
            if 'comm_part' in jsonfile:
                print("opening " + jsonfile)
                with io.open(jsonfile, 'r') as file:
                    data = json.load(file)
                    get_player_score_fromjson(data)
                    
#                     print(metadata)
personal_comm

In [36]:
result = dict()
# sumofpoints = 0
for key in personal_comm.keys():
    sumofpoints = 0
    for item in personal_comm[key]:
        sumofpoints += item[0]
#     print (key, sumofpoints, len(personal_comm[key]))
    result[key] = sumofpoints/len(personal_comm[key])

In [37]:
import operator
sorted_list = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [38]:
newdict = dict()
for x,y in sorted_list:
    newdict[y] = x

In [39]:
val_list = [x for y,x in sorted_list ]

## Selected players and their points <span style="color : darkblue"> for 20 matches</span>

In [40]:
for x in val_list[:11]:
    print (newdict[x], x)

mohammad saifuddin 0.19967692307692309
mashrafe mortaza 0.18356847826086953
mehidy hasan 0.15923958333333335
imrul kayes 0.13530298507462693
soumya sarkar 0.13002125
mohammad mithun 0.12362258064516132
liton das 0.11136666666666668
mushfiqur rahim 0.09877460317460317
tamim iqbal 0.06810204081632654
mahmudullah 0.06521333333333333
mustafizur rahman 0.05910400000000001


In [41]:
personal_comm['imrul kayes']

[[0.0, 'Imrul Kayes 4.'],
 [0.6124,
  'Shahin Newaz: "Saifuddin is a poor change in place of rubel... Rubel has the knack of bowling quick in a good length on a consistent basis where as saifuddin is more of a change of pace bowler..."\n\nSazedur Rahman: "I would love to see Mithun instead of Imrul, and Saifuddin instead of Rubel.'],
 [0.0,
  'Mohammad Mithun comes in for Imrul Kayes, and Mohammad Saifuddin replaces Rubel Hossain.'],
 [-0.4215,
  'Imrul Kayes brought back to rescue us in the Asia Cup, scored 328 runs in 3 innings against Zimbabwe and then suddenly 2 failures and he is back out.'],
 [-0.4215,
  'Imrul Kayes brought back to rescue us in the Asia Cup, scored 328 runs in 3 innings against Zimbabwe and then suddenly 2 failures and he is back out.'],
 [0.8622,
  'Reem Rahim: "I think BD management Took a great decision to left out Imrul kayes.They brought back Soumya Sarkar at 3 which is a great decision and Sarkar is always Agressive in those opening and Number 3 position..